In [1]:
###Importing the Important Libraries

import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
import os

from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau ,  ModelCheckpoint
from keras.applications.resnet50 import ResNet50, preprocess_input

Using TensorFlow backend.


In [3]:
###Constants

PATIENCE = 3
TRAIN_PATH= '/kaggle/input/10-monkey-species/training/training/'
VALID_PATH = '/kaggle/input/10-monkey-species/validation/validation/'

In [4]:
###Image Hyperparameters

IMAGE_SIZE = 224
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE,IMAGE_SIZE
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

In [5]:
###Creating the Model
from keras.applications.resnet50 import ResNet50, preprocess_input

input_shape = (IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_CHANNELS)

pre_trained_model = ResNet50(input_shape=input_shape, include_top=False, weights="imagenet",pooling='avg')

for layer in pre_trained_model.layers:
    layer.trainable = False
    
last_output = pre_trained_model.output

# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(last_output)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = Dense(10, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

94658560/94653016 [==============================] - 3s 0us/step


In [13]:
###Compiling the Model

model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [14]:
###Model Summary

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [15]:
###Callbacks

learning_rate_reduction = ReduceLROnPlateau(monitor='val_categorical_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

earlystop = EarlyStopping(patience=10)

callbacks = [earlystop, learning_rate_reduction]

In [16]:
###Hyperparameters

batch_size= 32
epochs = 5

In [17]:
###Generator
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
traingen = datagen.flow_from_directory(TRAIN_PATH, target_size=(224,224), batch_size=batch_size, class_mode='categorical')
validgen = datagen.flow_from_directory(VALID_PATH, target_size=(224,224), batch_size=batch_size, class_mode='categorical', shuffle=False)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [20]:
###Training the Model

history = model.fit_generator(
    traingen, 
    epochs=epochs,
    validation_data=validgen,
    validation_steps=len(validgen),
    steps_per_epoch=len(traingen),
    callbacks=callbacks
)

Epoch 1/5
35/35 [==============================] - 32s 915ms/step - loss: 0.1574 - categorical_accuracy: 0.9499 - val_loss: 0.1352 - val_categorical_accuracy: 0.9632
Epoch 2/5
35/35 [==============================] - 32s 907ms/step - loss: 0.1034 - categorical_accuracy: 0.9672 - val_loss: 0.0772 - val_categorical_accuracy: 0.9301
Epoch 3/5
35/35 [==============================] - 31s 891ms/step - loss: 0.1111 - categorical_accuracy: 0.9636 - val_loss: 0.3032 - val_categorical_accuracy: 0.9522

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/5
35/35 [==============================] - 32s 910ms/step - loss: 0.0637 - categorical_accuracy: 0.9800 - val_loss: 0.1043 - val_categorical_accuracy: 0.9559
Epoch 5/5
35/35 [==============================] - 31s 895ms/step - loss: 0.0297 - categorical_accuracy: 0.9927 - val_loss: 0.3012 - val_categorical_accuracy: 0.9522

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.


In [1]:
####Use abt 10 Epochs